# Single Modality

In [1]:
from models import *
from utils import *

ModuleNotFoundError: No module named 'torch'

In [ ]:
class MultiAttention(nn.Module):
    def __init__(self, spatial_dep, emb_dim, feat_dim=5, eeg=False, pos=None, n_class=2):
        super(MultiAttention, self ).__init__()
        
        if eeg:
            self.spatial_attention = RegionRNN_DEAP(emb_dim//2, 1, feat_dim, f_dim=feat_dim)
        else: 
            self.spatial_attention = SimpleRNN_DEAP(emb_dim, 1, feat_dim, f_dim=feat_dim)
            
        self.Classifier = nn.Sequential(
            nn.Linear(emb_dim, 64),
            nn.Dropout(0.25),
            nn.ReLU(True),
            nn.Linear(64, n_class),
            nn.Softmax(dim=1)
            )
    
        self.Regressor = nn.Sequential(
            nn.Linear(emb_dim, 64),
            nn.Dropout(0.25),
            nn.ReLU(True),
            nn.Linear(64, 1)
            )
        
        self.Discriminator = nn.Sequential(
            GradientReversal(),
            nn.Linear(emb_dim, 64),
            nn.Dropout(0.25),
            nn.ReLU(True),
            nn.Linear(64, 1),
            nn.Sigmoid()
            )
        
        
    def forward(self, x):
        b_size = x.shape[0]
        
        spatial_x = x.transpose(1,2)
        feat = self.spatial_attention(spatial_x)
        
        return feat

In [ ]:
'''Parameters'''
batch_size = 64
n_epoch = 1000

In [ ]:
def train_model(feat, label, participant, path_results, training_info, n_class, is_eeg=False, pos=None):
    EEG = EEGDataset(label=label, eeg=feat)
    Tot = {}
    session = np.load('dataset/seed_iv/session.npy')
    for p in tqdm(np.unique(participant)):
        idx = np.argwhere(participant==p).squeeze()
        np.random.shuffle(idx)
        #id_train = np.argwhere(participant!=p).squeeze()
        #id_test = idx        
        id_train = idx[:int(0.8*len(idx))]
        id_test = idx[int(0.8*len(idx)):]
        #id_train = np.argwhere(np.logical_and(participant==p, session<=16)).squeeze()
        #id_test = np.argwhere(np.logical_and(participant==p, session>16)).squeeze()
        np.random.shuffle(id_train)
        np.random.shuffle(id_test)
        Test = Subset(EEG, id_test)
        Train = Subset(EEG, id_train)

        Trainloader = DataLoader(Train, batch_size=batch_size, shuffle=False)
        Testloader = DataLoader(Test, batch_size=batch_size, shuffle=False)

        n_chan = feat.shape[2]
        f_dim = feat.shape[1]


        net = MultiAttention(spatial_dep=n_chan, emb_dim=64, feat_dim=f_dim, eeg=is_eeg, pos=pos, n_class=n_class).cuda()
        optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=0.001)

        writer = SummaryWriter('runs/rnn_phy_'+str(np.random.randint(0, 1000)))

        res = []
        for epoch in range(n_epoch):

            running_loss = 0.0
            evaluation = []
            t_cycle = iter(cycle(Testloader))
            net.train()
            for i, data in enumerate(Trainloader, 0):
                # get the inputs; data is a list of [inputs, labels]
                inputs_source, labels = data
                del data

                data = next(t_cycle)
                inputs_test, _ = data
                del data

                domain = torch.cat([torch.ones(inputs_source.shape[0]),
                                   torch.zeros(inputs_test.shape[0])]).cuda()
                inputs = torch.cat([inputs_source, inputs_test])

                # zero the parameter gradients
                optimizer.zero_grad()
                # forward + backward + optimize
                feat_ = net(inputs.to(torch.float32).cuda())
                domain_pred = net.Discriminator(feat_).squeeze()

                loss = torch.nn.functional.binary_cross_entropy(domain_pred, domain)

                outputs = net.Classifier(feat_[:inputs_source.shape[0]])

                label_loss = torch.nn.functional.cross_entropy(outputs, labels.to(torch.long).cuda())
                loss += label_loss
                loss.backward()
                optimizer.step()

                _, predicted = torch.max(outputs, 1)
                num_of_true = torch.sum(predicted.detach().cpu()==labels).numpy()
                mean = num_of_true/labels.shape[0]
                running_loss += label_loss.item()
                evaluation.append(mean)
            running_loss = running_loss/(i+1)
            running_acc = sum(evaluation)/len(evaluation)

            validation_loss = 0.0
            validation_acc = 0.0
            evaluation = []
            net.eval()
            for i, data in enumerate(Testloader, 0):
                inputs, labels = data
                del data
                outputs = net(inputs.to(torch.float32).cuda())
                outputs = net.Classifier(outputs)
                loss = torch.nn.functional.cross_entropy(outputs, labels.cuda())
                validation_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                num_of_true = torch.sum(predicted.detach().cpu()==labels).numpy()
                evaluation.append(num_of_true/labels.shape[0])
            validation_loss = validation_loss/(i+1)
            validation_acc = sum(evaluation)/len(evaluation)

            writer.add_scalar('Loss/Train', running_loss, epoch)
            writer.add_scalar('Loss/Test', validation_loss, epoch)
            writer.add_scalar('Acc/Train', running_acc, epoch)
            writer.add_scalar('Acc/Test', validation_acc, epoch)
            res.append((running_loss, running_acc, validation_loss, validation_acc))
            #if epoch%5==4:
            #    print('epoch %d \t training loss: %.3f - training acc: %.3f \t validation loss: %.3f - validation acc: %.3f'% (epoch+1, running_loss, running_acc, validation_loss, validation_acc))
        Tot['participant_'+str(p)] = np.asarray(res)
        np.save(os.path.join(path_results, training_info), Tot)

## PhyDAA Dataset

In [ ]:
'''Load File'''
label = np.load('dataset/phydaa/label.npy').astype(int)
n_class = len(np.unique(label))
participant = np.load('dataset/phydaa/participant.npy')
elec_pos = np.load('information/phydaa_eeg.npy')

### EEG

In [ ]:
train_model(feat=np.load('dataset/phydaa/feat_eeg.npy'), label=label, participant=participant, 
            path_results='res/ind/', training_info='eeg_phydaa_rnn_nodom', n_class=n_class, is_eeg=True, pos=None)

### Physiological

In [ ]:
train_model(feat=np.expand_dims(np.load('dataset/phydaa/feat_phy.npy'), 1), label=label, 
            participant=participant, path_results='res/ind/', training_info='feat_phydaa_rnn_nodom', n_class=n_class, is_eeg=False, pos=None)

## SEED IV

In [ ]:
'''Load File'''
label = np.load('dataset/seed_iv//label.npy').astype(int)
n_class = len(np.unique(label))
participant = np.load('dataset/seed_iv/participant.npy')
elec_pos = np.load('information/seed_iv_eeg.npy')

### EEG

In [ ]:
train_model(feat=np.load('dataset/seed_iv/feat_eeg.npy'), label=label, participant=participant, 
            path_results='res/dep/', training_info='eeg_seed_iv', n_class=n_class, is_eeg=True, pos=elec_pos)

### Physiological

In [ ]:
train_model(feat=np.expand_dims(np.load('dataset/seed_iv/feat_phy.npy'), 1), label=label, 
            participant=participant, path_results='res/dep/', training_info='feat_seed_iv', n_class=n_class, is_eeg=False, pos=None)

## DEAP

In [ ]:
'''Load File'''
label = np.load('dataset/deap/label.npy') - 4.5
n_class = 2 #len(np.unique(label))
participant = np.load('dataset/deap/participant.npy')

### EEG

In [ ]:
train_model(feat=np.load('dataset/deap/feat_eeg.npy'), label=(label[:, 0]>0).astype(int), participant=participant, 
            path_results='res/ind/', training_info='eeg_deap_valence', n_class=n_class, is_eeg=True, pos=None)

train_model(feat=np.load('dataset/deap/feat_eeg.npy'), label=(label[:, 1]>0).astype(int), participant=participant, 
            path_results='res/ind/', training_info='eeg_deap_arousal', n_class=n_class, is_eeg=True, pos=None)

### Physiological

In [ ]:
train_model(feat=np.load('dataset/deap/feat_phy.npy'), label=(label[:, 0]>0).astype(int), participant=participant, 
            path_results='res/ind/', training_info='feat_deap_valence', n_class=n_class, is_eeg=False, pos=None)

train_model(feat=np.load('dataset/deap/feat_phy.npy'), label=(label[:, 1]>0).astype(int), participant=participant, 
            path_results='res/ind/', training_info='feat_deap_arousal', n_class=n_class, is_eeg=False, pos=None)

In [ ]:
import IPython

IPython.Application.instance().kernel.do_shutdown(True)